#PubMedは論文検索サービスとして有用かどうか調べる

##目的
Google検索では論文以外のページもヒットしてしまうため、論文検索には適していない。  
論文検索のためのサービスはいくつか存在するが、医学分野の論文検索にはPubMedがよく用いられるらしい。  
今回は、PubMedが論文検索において本当に有用であるのかを調査する。

##方法
論文検索で想定される例を3例用意し、これらが達成できるかどうかで有用性を評価する。

1. 老化分野の基礎知識を得るために、レビュー論文を検索する
2. 川島隆太教授の研究を調べるために、彼の全ての論文を抽出する
3. 上記のうち、被引用数が上位10件の論文だけ選択する

これらの検索方法及び結果は、全てJupyter Notebookに記載する。  
したがって、上記の全ての検索はブラウザ上ではなくBioPythonで実行される。

##実行環境
* Google Chrome（バージョン: 80.0.3987.132）
* Google Colaboratory（ランタイム等は全てデフォルトのまま）

##実行手順

###0. BioPythonの準備

Google Colaboratory（以下、Colab）にはBioPythonが導入されていないので、まずはパッケージをインストールする。

In [0]:
!pip install biopython

     |████████████████████████████████| 2.3MB 2.8MB/s 


BioPythonをインポートする。  
スクレイピングを連続で行うと相手サーバーに負荷を与えてしまうため、休止時間を設けるためにsleepもインポートしておく。

In [0]:
from Bio import Entrez
from time import sleep

メールアドレスを入力しないとBioPythonが実行できないので、入力する。  
このメールアドレスはNCBI APIのエラー通知先として使用されるらしい。

In [0]:
Entrez.email = "emoriiin979@gmail.com"

###1. 老化分野の基礎知識を得るために、レビュー論文を検索する

老化分野を調べるので、キーワードは「aging」とした。  
レビュー論文のみを閲覧したいので、記事タイプは「Systematic Review」とした。  
その他、フルテキストであること、出版日が2018年4月以降であること、ヒトを対象としていることを条件に加えた。

In [0]:
term = "aging"
term += " AND Systematic Review[Publication Type]"
term += " AND Free full text[Filter]"
term += ' AND ("2018/04"[Date - Publication]: "3000/03"[Date - Publication])'
term += " AND Humans[Filter]"

BioPythonでのPubMed検索は今後も実行するので、実行しやすいように関数を作成しておく。  
上で設定した検索ワードを入力し、PubMed検索結果のPMIDを出力させる。

In [0]:
def get_ID(term, retmax=20):
    handle = Entrez.esearch(db="pubmed", term=term, retmax=retmax)
    record = Entrez.read(handle)
    return record["IdList"]

PubMed検索を実行する。  
本来なら204件の論文が出力されるが、それほどの数は不要なので、上位10件だけ出力するようにした。（retmax=10）

In [0]:
IdList = get_ID(term, retmax=10)

本当はBest Matchも実装したかったが、BioPythonでのやり方が分からないので断念した。  
レビュー論文を調べるだけなら、この時点でも達成可能である。

結果を見やすくするために、HTMLファイルを作成する。  
この機能はこの後も何度か利用するので、関数として定義しておく。

In [0]:
def make_HTML(IdList, outfile="out.html"):
    text = [
        "<!DOCTYPE html>",
        "<head>",
        '  <meta charset="UTF-8">',
        "  <title>Paper List</title>",
        "</head>",
        "<body>",
        "  <h1>Paper List</h1>"
    ]
    for pmid in IdList:
        url = "https://www.ncbi.nlm.nih.gov/pubmed/" + pmid + "/"
        line = '  <p><a href="' + url + '" target="_blank">' + url + "</a></p>"
        text.append(line)
    text.append("</body>")
    with open(outfile, "w") as o:
        for t in text:
            print(t, file=o)

結果をHTMLファイルに出力した。  
ブラウザでファイルを開くと、検索結果の10報の論文のアドレスが表示される。

In [0]:
make_HTML(IdList, outfile="res1.html")
print(IdList)

['32078523', '31860946', '31814714', '31752149', '31739647', '31722884', '31695349', '31690308', '31676005', '31635264']


###2. 川島隆太教授の研究を調べるために、彼の全ての論文を抽出する

川島隆太教授は東北大学で加齢医学の研究を行っている人物であり、「脳トレ」で一般によく知られている。  
今回はPubMedのケーススタディとして、有名人でかつ老化の研究を行っている人として、彼の研究内容を調査することにした。

In [0]:
term = "Kawashima Ryuta[Author]"
term += " AND Tohoku[Affiliation]"

著者名（Author）と所属機関（Affiliation）のタグを用いる。  
ブラウザ版のPubMedでは、件数が「298件」となった。（2020/03/23に検索）  
全ての論文を出力したいので、retmaxは300に設定した。

In [0]:
handle = Entrez.esearch(db="pubmed", term=term, retmax=300)
record = Entrez.read(handle)
IdList = record["IdList"]

IdListの件数を確認する。

In [0]:
print(IdList[0:4])
print(len(IdList))

['32120794', '32021932', '32010024', '31980740']
298


上位4件のPMIDと、出力件数ともに、ブラウザ版PubMedのものと相違なかった。

###3. 被引用数が上位10件の論文だけ選択する

2.で抽出された論文数は、298件にも及んだ。  
これほどの論文数を全て調査するのは大変なので、重要そうな論文だけを調べたい。  
今回は、被引用数が大きい論文だけを選択することにした。

被引用数はPubMedで調べる方法が分からなかったので、外部サービスを用いて調べた。  
外部サービスの出力結果はスクレイピングで取得する。  
そのためのrequestsモジュールをインポートする。

In [0]:
import requests

被引用数はSemantic Scholarが提供している。  
本サービスでは被引用数を出力するAPIが提供されているので、それを活用する。  
APIの出力結果をrequestsで取得し、その中から被引用文献（citations）のリストを抽出し、数をカウントする。  
この処理は298件全ての論文に対して行われるため、関数として定義しておく。

In [0]:
def get_n_cites(pmid):
    url = "https://api.semanticscholar.org/v1/paper/PMID:" + pmid
    result = requests.get(url).json()
    n_cites = len(result["citations"])
    return n_cites

論文のPMIDと被引用数を結び付けて記録するため、そのためのPandasデータフレームを定義する。  
列は「PMID」と「被引用数（n_cites）」であり、各行にそれぞれの論文のPMIDと被引用数が格納される。

In [0]:
import pandas as pd

In [0]:
df = pd.DataFrame(columns=["PMID", "n_cites"])

IdListに格納されている全てのPMIDに対して、関数get_n_citesを適用する。  
出力結果はデータフレームdfに追記されていく。  
連続スクレイピングによる負荷を軽減するため、sleep関数も使用する。

In [0]:
for id in IdList:
    n_cites = get_n_cites(id)
    df = df.append(pd.Series([id, n_cites], index=df.columns), ignore_index=True)
    sleep(1)

被引用数が上位10件の論文を選択する。

In [0]:
df_sorted = df.sort_values("n_cites", ascending=False)[0:10]
df_sorted

,PMID,n_cites
239,20203189,304
258,18223623,186
207,22253758,153
286,14980557,139
230,21111830,125
238,20226253,116
279,15588605,113
240,20171286,104
218,21886781,98
287,14741309,98


出力結果をHTMLファイルに入力する。

In [0]:
make_HTML(list(df_sorted["PMID"]), outfile="res3.html")

##考察

検索の結果出力された論文のうち上位10件の中身を確認し、目的の論文が得られているかを検討する。

###1. 老化分野の基礎知識を得るために、レビュー論文を検索する

####論文の概要
PMID: 32078523  
椎間板ヘルニアによる慢性腰痛の症例レビューに基づいた、新規の患者に対する作業療法介入の成功例

PMID: 31860946  
前立腺がんの疼痛に対する薬物療法の有効性・安全性のメタ分析

PMID: 31814714  
体系的レビューの文献調査による、根治的前立腺摘除術後の尿失禁に対する骨盤底筋トレーニングの有効性評価

PMID: 31752149  
認知症の有無が口腔衛生に及ぼす影響を文献調査

PMID: 31722884  
大腸内視鏡検査の検査間隔を決定するための、内視鏡検査と大腸がん有病率の文献調査

PMID: 31695349  
アルツハイマー型認知症患者に対する虐待行為の調査と防止策の検討

PMID: 31690308  
多発性疾患の発生を感知し適切なサービスへ繋げることができているかを調査

PMID: 31676005  
認知症の介護者に対するe-ラーニング教育の活用例の調査

PMID: 31635264  
腸内細菌叢から産生される短鎖脂肪酸と肥満の関係性を調査

PMID: 31627427  
加齢に伴うサルコペニア（筋肉喪失）に対するロイシン補充の有効性調査

論文の形式を「Systematic Review」に指定したため、ほとんどの論文で文献調査が行われ、それに基づいた結論が述べられていた。Systematic Reviewではランダム盲検テストなど信頼性の高いテスト方法が用いられている文献のみが資料として利用されるため、文献調査の結果は信頼性が高いといえる。医学分野に関しては、ReviewよりもSystemaric Reviewの方が質の高い情報が得られるかもしれない。  
一方で、キーワードを「aging」としか入力しなかったせいか、扱われる内容が多岐に渡ってしまった。そもそも老化そのものに関する論文は一つも存在せず、ほとんどが高齢者の疾患に関する内容であった。本当に知りたい情報を出力させるには、適切なキーワードを選択しなければならない。老化の教科書などを参考にして、現時点で流行しているキーワードを調べておく必要があるかもしれない。

###2. 川島隆太教授の研究を調べるために、彼の全ての論文を抽出する

####論文の著者名（「Kawashima」のみ）と所属機関名
PMID: 32120794  
Ryuta Kawashima, Tohoku University

PMID: 32021932  
Ryuta Kawashima, Tohoku University

PMID: 32010024  
Ryuta Kawashima, Tohoku University

PMID: 31980740  
Ryuta Kawashima, Tohoku University

PMID: 31978544  
Ryuta Kawashima, Tohoku University

PMID: 31936294  
Ryuta Kawashima, Tohoku University

PMID: 31932477  
Ryuta Kawashima, Tohoku University

PMID: 31902739  
Ryuta Kawashima, Tohoku University

PMID: 31888419  
Ryuta Kawashima, Tohoku University

PMID: 31866843  
Ryuta Kawashima, Tohoku University

検索結果の上位10件では、全ての論文で「Ryuta Kawashima」が著者として存在し、いずれも所属機関名が「Tohoku University」であった。東北大学に川島隆太教授と同姓同名の人間がいない限り、この著者は全て川島教授本人であると考えられる。  
298件すべてを調べたわけではないので確かではないが、検索結果のほぼ全てが川島教授が関与していると考えてよいと結論付けた。現在知る限りでは著者の指定にはAuthorタグを用いるのが最も簡単であるため、今後も研究室の論文探しにはAuthorタグで検索したい。

###3. 被引用数が上位10件の論文だけ選択する

PMID: 20203189  
ワーキングメモリに影響を与える「白質の統合性」がトレーニングによって変化するかどうかを調査

PMID: 18223623  
BMI（肥満度）と脳灰白質量の相関関係の調査

PMID: 22253758  
脳トレゲームが高齢者の認知機能に与える影響を評価

PMID: 14980557  
脳fMRIの血中酸素レベル依存型応答（BOLD応答）から血行力学パラメータを求めるためのモデル式の改良

PMID: 21111830  
ワーキングメモリの非活性化による拡散的思考と創造性向上の関係性をfMRIで調査

PMID: 20226253  
脳内の局所的な灰白質量が創造性テストの結果に与える影響をVBMで調査

PMID: 15588605  
自分の顔を認識するために活性化される脳の部位をfMRIで調査

PMID: 20171286  
脳内の局所的な白質統合性（構造的接続性）と創造力の関係性をDTIで調査

PMID: 21886781  
ワーキングメモリのトレーニングが脳内の灰白質量に与える影響をVBMで調査

PMID: 14741309  
簡単な暗算で活性化される脳部位を子供と大人で比較

川島教授が関与する論文で印象的なのは、fMRIなどの脳イメージングが必ず出現する点であった。想像力テストや計算問題などの課題を被験者に与えて、その時の脳シグナルをfMRI、VBM、DTIで検出している。2報目だけは川島教授が筆頭・最終著者ではなく、この論文だけBMI（肥満度）を対象としているなど毛色が他とは異なるが、ここでも灰白質量を測定するためにVBMが使われているため、おそらく脳イメージングの所だけ部分的に関与したのだと予想される。  
ただし、被引用数が多い論文は比較的古いものが多いため、最近の研究室事情を反映していない可能性がある。被引用数でフィルタリングするだけでなく、直近の論文も併せて読むことで、より正確な研究内容を把握できるのではないかと考えられる。

以上。
2020年3月28日　えもり  
2020年4月1日　「目的」を追記